In [113]:
import os
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline     
sns.set(color_codes=True)

In [114]:
# TODO: change this to yours ...
DATA_DIR = "/Users/kwongtszkong/Desktop/STAT3612 Statistical ML/Group Project/Stat3612_Project_datasource"

# read 3 csv files and 1 pkl file
train_csv_file = os.path.join(DATA_DIR, "train.csv")
val_csv_file = os.path.join(DATA_DIR, "valid.csv")
test_csv_file = os.path.join(DATA_DIR, "test.csv")
ehr_pkl_file = os.path.join(DATA_DIR, "ehr_preprocessed_seq_by_day_cat_embedding.pkl")

train_df = pd.read_csv(train_csv_file)
val_df = pd.read_csv(val_csv_file)
test_df = pd.read_csv(test_csv_file)

with open(ehr_pkl_file, 'rb') as f:
    ehr_data = pd.read_pickle(f)

Initialize the X_train with mapping features

In [115]:

# Extract EHR features and IDs
ehr_features = ehr_data["feat_dict"]
ehr_ids = list(ehr_features.keys())

# Convert to DataFrame (each row is an admission)
ehr_df = pd.DataFrame({
    "id": ehr_ids,
    "ehr_matrix": [ehr_features[id] for id in ehr_ids]
})

# Merge with labels from train/val/test DataFrames
def merge_labels(df, ehr_df):
    return df[["id", "readmitted_within_30days"]].merge(ehr_df, on="id", how="inner")

train_ehr = merge_labels(train_df, ehr_df)
val_ehr = merge_labels(val_df, ehr_df)

def aggregate_ehr(df):
    # Aggregate features (example: mean over days)
    df["ehr_mean"] = df["ehr_matrix"].apply(lambda x: np.mean(x, axis=0))
    
    # Convert to numpy arrays
    X = np.stack(df["ehr_mean"].values)
    y = df["readmitted_within_30days"].astype(int).values
    
    return X, y

X_train, y_train = aggregate_ehr(train_ehr)
X_val, y_val = aggregate_ehr(val_ehr)
print(X_train.shape)
print(X_train)



(49451, 171)
[[52.  1.  6. ...  0.  0.  0.]
 [52.  1.  6. ...  0.  0.  0.]
 [52.  1.  6. ...  0.  0.  0.]
 ...
 [91.  1.  6. ...  0.  0.  1.]
 [69.  0.  6. ...  0.  0.  0.]
 [69.  0.  6. ...  0.  0.  0.]]


Remove the non-informative features

In [125]:
from scipy.stats import pointbiserialr 

# Assuming X_train is your feature matrix (n_samples × n_features)
std_devs = np.std(X_train, axis=0)

# Get feature names from ehr_data (adjust if your structure differs)
feature_names = ehr_data["feature_cols"]  # Or ehr_data["feature_cols"] if available

# Create a DataFrame for analysis
std_df = pd.DataFrame({
    "Feature": feature_names,
    "Std_Dev": std_devs
})




# Initialize storage
correlations = []
p_values = []

# Calculate correlation for each feature
for i in range(X_train.shape[1]):
    if np.std(X_train[:, i]) == 0:  # Skip constant features
        correlations.append(0)
        p_values.append(1)
    else:
        corr, pval = pointbiserialr(X_train[:, i], y_train)
        correlations.append(corr)
        p_values.append(pval)

# Create results DataFrame
corr_df = pd.DataFrame({
    "Feature": feature_names,
    "Correlation": correlations,
    "P-value": p_values
})

# Sort by absolute correlation strength
corr_df["Abs_Correlation"] = np.abs(corr_df["Correlation"])
corr_df = corr_df.sort_values("Abs_Correlation", ascending=True)
print(corr_df[["Feature", "Correlation", "P-value"]].head(20))  # Top 20 features


                                               Feature  Correlation   P-value
44                                             M50-M54     0.000000  1.000000
80                                             K40-K46     0.000000  1.000000
39                                             A20-A28     0.000000  1.000000
17                                             N00-N08     0.000000  1.000000
34                                             E70-E88     0.000000  1.000000
72                                             J00-J06     0.000000  1.000000
111                              Basophils Joint Fluid     0.000000  1.000000
50                                             N25-N29     0.000000  1.000000
49                                             J30-J39     0.000000  1.000000
93                                             R90-R94     0.000000  1.000000
51                                             Q65-Q79     0.000000  1.000000
5                                              O85-O92     0.000

In [117]:

# Function to analyze zero-value ratio in features
def analyze_zeros(ehr_data, feature_cols, threshold=0.5):

    all_zeros = []
    for feature_name, feature_idx in zip(feature_cols, range(len(feature_cols))):
        zeros_count = np.sum([np.sum(matrix[:, feature_idx] == 0) for matrix in ehr_data["feat_dict"].values()])
        total_values = sum([matrix.shape[0] for matrix in ehr_data["feat_dict"].values()])
        zero_ratio = zeros_count / total_values
        if zero_ratio > threshold:
            all_zeros.append((feature_name, zero_ratio))
    return pd.DataFrame(all_zeros, columns=["Feature", "Zero_Ratio"])

# Analyze zero-value ratio for each feature category
demo_zero_df = analyze_zeros(ehr_data, ehr_data["demo_cols"])
icd_zero_df = analyze_zeros(ehr_data, ehr_data["icd_cols"])
lab_zero_df = analyze_zeros(ehr_data, ehr_data["lab_cols"])
med_zero_df = analyze_zeros(ehr_data, ehr_data["med_cols"])

# Combine all zero-value data
zero_df = pd.concat([demo_zero_df, icd_zero_df, lab_zero_df, med_zero_df])

# Merge zero-value data with std and correlation data
combined_df = pd.merge(pd.merge(std_df, corr_df, on='Feature'), zero_df, on='Feature', how='left')



clinically_relevant1 = [
    'E70-E88', 'N00-N08', 'N17-N19', 'I30-I52', 'J40-J47',
    'A00-A09', 'J20-J22', 'Basophils Blood', 'Eosinophils Blood',
    'pH Urine', 'ANTIINFLAM.TUMOR NECROSIS FACTOR INHIBITING AGENTS',
    'ANTIPARASITICS'
]
clinically_relevant = [
    # Labs
    'Creatinine Blood', 'Hemoglobin Blood', 'Hematocrit Blood',
    'Potassium Blood', 'Sodium Blood', 'Glucose Blood',
    'Troponin T Blood', 'Platelet Count Blood', 'Eosinophils Blood',
    'pH Urine', 'pO2 Blood', 'pCO2 Blood', 'Anion Gap Blood',
    
    # ICD-10
    'I10-I16', 'N17-N19', 'J09-J18', 'E70-E88', 'I30-I52',
    'J40-J47', 'B20-B20',
    
    # Drugs
    'ANTICOAGULANTS', 'ANTIBIOTICS', 'IMMUNOSUPPRESSANTS',
    'ANTIINFLAM.TUMOR NECROSIS FACTOR INHIBITING AGENTS'
]



/var/folders/79/xhsm8rlj5fz4b46rjy_ls9t40000gn/T/ipykernel_80528/3046049645.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  zero_df = pd.concat([demo_zero_df, icd_zero_df, lab_zero_df, med_zero_df])


In [118]:

thresholds = {
    'zero_ratio': {
        'icd': 0.95,    # ICD codes often sparse
        'med': 0.95,    # Medications often sparse
        'lab': 0.95,    # Labs should rarely be zero
        'demo': 0.9     # Demographics rarely zero
    },
    'std_dev': 0.05,    # Only for continuous features
    'correlation': {
        'min_abs_corr': 0.05,
        'max_pvalue': 0.05
    }
}

# Categorize features
feature_types = {
    'icd': ehr_data["icd_cols"],
    'med': ehr_data["med_cols"],
    'lab': ehr_data["lab_cols"],
    'demo': ehr_data["demo_cols"]
}

# Initialize storage
features_to_remove = []

# Check each feature type separately
for ftype, cols in feature_types.items():
    for feature in cols:
        row = combined_df[combined_df['Feature'] == feature].iloc[0]
        
        # Skip binary features for variance check
        check_variance = ftype not in ['icd']
        
        # Apply type-specific rules
        if (row['Zero_Ratio'] > thresholds['zero_ratio'][ftype]) and \
           (not check_variance or row['Std_Dev'] < thresholds['std_dev']) and \
           (np.abs(row['Correlation']) < thresholds['correlation']['min_abs_corr']) and \
           (row['P-value'] > thresholds['correlation']['max_pvalue']):
            features_to_remove.append(feature)

    
filtered_df = combined_df[~combined_df['Feature'].isin(clinically_relevant)]
# Generate report
removal_df = filtered_df[filtered_df['Feature'].isin(features_to_remove)].sort_values(
    by=['Zero_Ratio', 'Std_Dev', 'Abs_Correlation'],
    ascending=[False, True, True]
)

print(f"\nRecommended features to remove ({len(removal_df)} total):")
print(removal_df[['Feature', 'Zero_Ratio', 'Std_Dev', 'Correlation', 'P-value']])
print("Recommended removal features list:")
print(removal_df['Feature'].tolist())


Recommended features to remove (42 total):
                   Feature  Zero_Ratio   Std_Dev  Correlation   P-value
54                 K65-K68    0.999950  0.015576    -0.008536  0.057676
24                 R40-R46    0.999950  0.102142    -0.003603  0.423045
8                  M80-M85    0.999909  0.007789    -0.004268  0.342633
120        Basophils Blood    0.999877  0.017834    -0.000048  0.991520
29                 B25-B34    0.999877  0.029816     0.004573  0.309156
26                 N20-N23    0.999863  0.008993    -0.004928  0.273169
73                 N60-N65    0.999831  0.025824     0.006283  0.162356
47                 G89-G99    0.999813  0.182042     0.007699  0.086893
44                 M50-M54    0.999786  0.000000     0.000000  1.000000
111  Basophils Joint Fluid    0.999758  0.000000     0.000000  1.000000
147         CONTRACEPTIVES    0.999758  0.010055    -0.005509  0.220519
57                 B65-B83    0.999735  0.006359    -0.003484  0.438443
80                 K

In [119]:

# Assuming 'combined_df' is your main DataFrame containing all the feature statistics
# 'clinically_relevant' is a list of features you've decided must not be removed
# 'features_to_remove' is your list of features identified for potential removal

# Filter to find features not in the removal list and that are clinically relevant
not_removal_df = combined_df[
    (~combined_df['Feature'].isin(features_to_remove)) | 
    (combined_df['Feature'].isin(clinically_relevant))
]

# Sort the DataFrame by specified criteria (if needed)
not_removal_df = not_removal_df.sort_values(
    by=['Zero_Ratio', 'Std_Dev', 'Correlation'],
    ascending=[False, True, True]
)

# Display the DataFrame containing the features not recommended for removal
print(f"\nFeatures not recommended for removal ({len(not_removal_df)} total):")
print(not_removal_df[['Feature', 'Zero_Ratio', 'Std_Dev', 'Correlation', 'P-value']])


Features not recommended for removal (129 total):
                                               Feature  Zero_Ratio    Std_Dev  Correlation        P-value
126                                  Monocytes Ascites    0.999982   0.015951    -0.093610   1.186603e-96
35                                             K20-K31    0.999982   0.053321    -0.025698   1.094875e-08
162                                        ANESTHETICS    0.999982   2.659306     0.098794  1.742035e-107
42                                             R70-R79    0.999968   0.056435    -0.021665   1.448250e-06
169                                           VITAMINS    0.999968   1.175168     0.147812  1.524469e-239
92                                             N40-N53    0.999964   0.013489     0.017510   9.856692e-05
75                                             K35-K38    0.999959   0.007789     0.014217   1.569526e-03
60                                             N17-N19    0.999959   0.020107     0.005686   2.060585

In [120]:
# 1. Get indices of features to remove
features_to_remove = removal_df['Feature'].tolist()
all_features = ehr_data["feature_cols"]
remove_indices = [i for i, feature in enumerate(all_features) 
                 if feature in features_to_remove]

# 2. Function to remove features
def remove_features(X, remove_indices):
    return np.delete(X, remove_indices, axis=1)

# 3. Apply to both training and validation sets
X_train_filtered = remove_features(X_train, remove_indices)
X_val_filtered = remove_features(X_val, remove_indices)

# 4. Get remaining feature names
remaining_features = [f for i, f in enumerate(all_features) 
                     if i not in remove_indices]

# 5. Verification
print(f"Removed {len(remove_indices)} features.")
print(f"Original shape: {X_train.shape} -> New shape: {X_train_filtered.shape}")
print("\nSample removed features:", features_to_remove[:5])
print("\nSample remaining features:", remaining_features[:5])

Removed 42 features.
Original shape: (49451, 171) -> New shape: (49451, 129)

Sample removed features: ['K65-K68', 'R40-R46', 'M80-M85', 'Basophils Blood', 'B25-B34']

Sample remaining features: ['age', 'gender', 'ethnicity', 'Y90-Y99', 'G30-G32']


In [121]:

from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report


# 2. Scale features (important for logistic regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_filtered)
X_val_scaled = scaler.transform(X_val_filtered)

# 3. Feature selection with Logistic Regression
# (Using L1 penalty for automatic feature selection)
selector = SelectFromModel(
    LogisticRegression(
        penalty='l1',         # Lasso regularization
        solver='liblinear',   # Works well with L1
        class_weight='balanced',
        random_state=42,
        max_iter=1000
    ),
    threshold='median'       # Keep features with importance > median
)

# Fit selector
selector.fit(X_train_scaled, y_train)

# Get selected features
selected_mask = selector.get_support()
selected_features = np.array(remaining_features)[selected_mask]
X_train_selected = selector.transform(X_train_scaled)
X_val_selected = selector.transform(X_val_scaled)

# 4. Train final model on selected features
final_model = LogisticRegression(
    penalty='l2',            # Switch to Ridge for final model
    class_weight='balanced',
    random_state=42,
    max_iter=1000
)
final_model.fit(X_train_selected, y_train)

# 5. Evaluate
print("\n=== Feature Selection Results ===")
print(f"Original features: {X_train_filtered.shape[1]}")
print(f"Selected features: {X_train_selected.shape[1]}")
print("\nTop 10 selected features:")
for feat in selected_features[:10]:
    print(f"- {feat}")

print("\nValidation Performance:")
y_pred = final_model.predict(X_val_selected)
print(classification_report(y_val, y_pred))

# 6. (Optional) Get feature importance
coef_df = pd.DataFrame({
    'Feature': selected_features,
    'Coefficient': final_model.coef_[0],
    'Abs_Coef': np.abs(final_model.coef_[0])
}).sort_values('Abs_Coef', ascending=False)




=== Feature Selection Results ===
Original features: 129
Selected features: 65

Top 10 selected features:
- age
- Y90-Y99
- A00-A09
- F01-F09
- F60-F69
- I80-I89
- I95-I99
- K55-K64
- J60-J70
- G35-G37

Validation Performance:
              precision    recall  f1-score   support

           0       0.85      0.75      0.80     12800
           1       0.42      0.58      0.48      3921

    accuracy                           0.71     16721
   macro avg       0.64      0.67      0.64     16721
weighted avg       0.75      0.71      0.73     16721



In [122]:
# After running your feature selection code...

# Print all selected features with their coefficients
print("\n=== All Selected Features ===")
print(f"Total selected: {len(selected_features)} features\n")

# Create a DataFrame for better visualization
selected_features_df = pd.DataFrame({
    'Feature': selected_features,
    'Coefficient': final_model.coef_[0],
    'Abs_Coefficient': np.abs(final_model.coef_[0]),
    'Direction': np.where(final_model.coef_[0] > 0, 'Positive', 'Negative')
})

# Sort by absolute coefficient value (most important first)
selected_features_df = selected_features_df.sort_values('Abs_Coefficient', ascending=False)

# Print all selected features
pd.set_option('display.max_rows', None)  # Show all rows
print(selected_features_df[['Feature', 'Coefficient', 'Direction']])
pd.reset_option('display.max_rows')  # Reset display options

# Alternative simple print (just names)
print("\nAs simple list:")
for i, feat in enumerate(selected_features, 1):
    print(f"{i}. {feat}")


=== All Selected Features ===
Total selected: 65 features

                         Feature  Coefficient Direction
0                            age     0.436251  Positive
21                       G00-G09    -0.383844  Negative
30                 Lactate Blood    -0.351508  Negative
14                       F20-F29    -0.341737  Negative
20                       E50-E64    -0.309881  Negative
18                       C15-C26    -0.257551  Negative
63            SEDATIVE/HYPNOTICS     0.251135  Positive
9                        G35-G37    -0.239004  Negative
8                        J60-J70    -0.227523  Negative
28                     pO2 Blood    -0.225913  Negative
4                        F60-F69    -0.207159  Negative
1                        Y90-Y99    -0.203559  Negative
19                       C7A-C7A    -0.202372  Negative
23                       J20-J22    -0.190946  Negative
42                    pCO2 Blood    -0.188475  Negative
41           Lymphocytes Pleural    -0.18583

In [123]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler

# Assuming you have:
# selected_features = list of feature names selected by your model
# remaining_features = list of all feature names after initial filtering
# X_train_filtered, X_val_filtered = your pre-filtered feature matrices

# 1. Get indices of selected features
selected_indices = [i for i, feat in enumerate(remaining_features) 
                   if feat in selected_features]

# 2. Extract selected features from training/validation sets
X_train_selected = X_train_filtered[:, selected_indices]
X_val_selected = X_val_filtered[:, selected_indices]
print(f"Selected features shape: {X_train_selected.shape}")

# 3. Scale the data (important for logistic regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_val_scaled = scaler.transform(X_val_selected)

# 4. Train logistic regression
logreg = LogisticRegression(
    penalty='l2',
    class_weight='balanced',
    random_state=42,
    max_iter=1000
)
logreg.fit(X_train_scaled, y_train)

# 5. Predict probabilities for AUC calculation
y_train_probs = logreg.predict_proba(X_train_scaled)[:, 1]
y_val_probs = logreg.predict_proba(X_val_scaled)[:, 1]

# 6. Calculate AUC scores
train_auc = roc_auc_score(y_train, y_train_probs)
val_auc = roc_auc_score(y_val, y_val_probs)

print(f"Number of selected features: {len(selected_features)}")
print(f"Train AUC: {train_auc:.4f}")
print(f"Validation AUC: {val_auc:.4f}")

# 7. Additional metrics (optional)
print("\nClassification Report:")
print(classification_report(y_val, logreg.predict(X_val_scaled)))

# 8. Feature importance analysis (optional)
coef_df = pd.DataFrame({
    'Feature': selected_features,
    'Coefficient': logreg.coef_[0],
    'Abs_Coefficient': np.abs(logreg.coef_[0])
}).sort_values('Abs_Coefficient', ascending=False)


Selected features shape: (49451, 65)
Number of selected features: 65
Train AUC: 0.7925
Validation AUC: 0.7107

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.75      0.80     12800
           1       0.42      0.58      0.48      3921

    accuracy                           0.71     16721
   macro avg       0.64      0.67      0.64     16721
weighted avg       0.75      0.71      0.73     16721



In [124]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Assuming you've already run your feature selection code above...

def calculate_feature_stats(X, y, feature_names):
    """Calculate comprehensive feature statistics"""
    stats_list = []
    
    for i, feat in enumerate(feature_names):
        # Get feature values
        feat_values = X[:, i]
        
        # Calculate statistics
        zero_rate = np.mean(feat_values == 0) * 100
        std_dev = np.std(feat_values)
        
        # Correlation and p-value (point-biserial correlation for binary y)
        corr, p_value = stats.pointbiserialr(feat_values, y)
        
        stats_list.append({
            'Feature': feat,
            'Zero_Rate(%)': zero_rate,
            'Std_Dev': std_dev,
            'Correlation': corr,
            'P_Value': p_value
        })
    
    return pd.DataFrame(stats_list)

# 1. Calculate all statistics for selected features
feature_stats_df = calculate_feature_stats(
    X_train_selected, 
    y_train, 
    selected_features
)

# 2. Merge with coefficient information
coef_df = pd.DataFrame({
    'Feature': selected_features,
    'Coefficient': final_model.coef_[0],
    'Abs_Coef': np.abs(final_model.coef_[0])
})

full_stats_df = pd.merge(coef_df, feature_stats_df, on='Feature').sort_values('Abs_Coef', ascending=False)

# 3. Print comprehensive report
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', None)

print("\n=== COMPREHENSIVE FEATURE STATISTICS ===")
print(f"Selected {len(selected_features)} features from {X_train_filtered.shape[1]} original features")
print(full_stats_df.round(4))

# 4. Print validation performance
print("\n\n=== VALIDATION PERFORMANCE ===")
y_pred = final_model.predict(X_val_selected)
print(classification_report(y_val, y_pred))




=== COMPREHENSIVE FEATURE STATISTICS ===
Selected 65 features from 129 original features
                         Feature  Coefficient  Abs_Coef  Zero_Rate(%)  Std_Dev  Correlation  P_Value
0                            age       0.4363    0.4363        0.0000  16.4603       0.0788   0.0000
21                       G00-G09      -0.3838    0.3838       99.8402   0.0399      -0.0219   0.0000
30                 Lactate Blood      -0.3515    0.3515        0.1982   0.1321      -0.3245   0.0000
14                       F20-F29      -0.3417    0.3417       99.8584   0.0407      -0.0201   0.0000
20                       E50-E64      -0.3099    0.3099       99.8746   0.0514      -0.0183   0.0000
18                       C15-C26      -0.2576    0.2576       99.8827   0.0342      -0.0188   0.0000
63            SEDATIVE/HYPNOTICS       0.2511    0.2511       45.8332   5.0183       0.1020   0.0000
9                        G35-G37      -0.2390    0.2390       99.9090   0.0302      -0.0165   0.0002
8